<a href="https://colab.research.google.com/github/KorStats/2022project/blob/main/%EC%9E%90%EB%8F%99labeling/160%EC%9E%A5%2Byolov5x%2Bbatch8%2Bepoch30_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글드라이브에서 파일 읽어오기

### 각자 이용할 데이터들을 images와 labels를 구분해서 zip 파일로 변환하여 드라이브에 저장 (images.zip, labels.zip 파일이 있어야함)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 2022project	    images_1730.zip   labels_1730.zip  'My Drive'
'Colab Notebooks'   images.zip	      labels.zip


In [3]:
!unzip /content/gdrive/MyDrive/images.zip -d /content/dataset

Archive:  /content/gdrive/MyDrive/images.zip
  inflating: /content/dataset/20220916_103619955.jpg  
  inflating: /content/dataset/20220916_105735553.jpg  
  inflating: /content/dataset/20220916_131619211.jpg  
  inflating: /content/dataset/20220916_171930418.jpg  
  inflating: /content/dataset/20220919_225825363.jpg  
  inflating: /content/dataset/20220919_231820473.jpg  
  inflating: /content/dataset/20220919_233039243.jpg  
  inflating: /content/dataset/20220919_234831665.jpg  
  inflating: /content/dataset/20220920_000250117.jpg  
  inflating: /content/dataset/20220920_002753658.jpg  
  inflating: /content/dataset/20220920_005456751.jpg  
  inflating: /content/dataset/20220920_011546170.jpg  
  inflating: /content/dataset/20220920_012637916.jpg  
  inflating: /content/dataset/20220920_013432931.jpg  
  inflating: /content/dataset/20220920_020037861.jpg  
  inflating: /content/dataset/20220920_021643168.jpg  
  inflating: /content/dataset/20220920_022746250.jpg  
  inflating: /conten

In [4]:
!unzip /content/gdrive/MyDrive/labels.zip -d /content/dataset

Archive:  /content/gdrive/MyDrive/labels.zip
  inflating: /content/dataset/20220916_103619955.txt  
  inflating: /content/dataset/20220916_105735553.txt  
  inflating: /content/dataset/20220916_131619211.txt  
  inflating: /content/dataset/20220916_171930418.txt  
  inflating: /content/dataset/20220919_225825363.txt  
  inflating: /content/dataset/20220919_231820473.txt  
  inflating: /content/dataset/20220919_233039243.txt  
  inflating: /content/dataset/20220919_234831665.txt  
  inflating: /content/dataset/20220920_000250117.txt  
  inflating: /content/dataset/20220920_002753658.txt  
  inflating: /content/dataset/20220920_005456751.txt  
  inflating: /content/dataset/20220920_011546170.txt  
  inflating: /content/dataset/20220920_012637916.txt  
  inflating: /content/dataset/20220920_013432931.txt  
  inflating: /content/dataset/20220920_020037861.txt  
  inflating: /content/dataset/20220920_021643168.txt  
  inflating: /content/dataset/20220920_022746250.txt  
  inflating: /conten

# YOLOV5 모델 시작

In [5]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 12886, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 12886 (delta 23), reused 28 (delta 18), pack-reused 12844
Receiving objects: 100% (12886/12886), 13.01 MiB | 3.81 MiB/s, done.
Resolving deltas: 100% (8856/8856), done.


In [6]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 12.1 MB/s 


In [10]:
%cd / 
from glob import glob

img_list1=glob('/content/dataset/*.jpg')
img_list2=glob('/content/dataset/*.JPG')
img_list=img_list1+img_list2
print(len(img_list))

/
160


In [11]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list=train_test_split(img_list, test_size=0.1, random_state=0)
print(len(train_img_list), len(val_img_list))

144 16


In [12]:
with open('/content/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list)+'\n')

with open('/content/dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list)+'\n')

In [13]:
import yaml

dataform= {'train': '/content/dataset/train.txt', 'val':'/content/dataset/val.txt', 'nc': 1, 'names' : ['crop']}

with open('/content/dataset/data.yaml', 'w') as f:
    yaml.dump(dataform, f)

%cat /content/dataset/data.yaml

names:
- crop
nc: 1
train: /content/dataset/train.txt
val: /content/dataset/val.txt


In [14]:
%cd /content/yolov5/

!python train.py --img 640 --batch 8 --epochs 30 --data /content/dataset/data.yaml --cfg ./models/yolov5x.yaml --weights yolov5x.pt --name grain_results --single-cls

/content/yolov5
train: weights=yolov5x.pt, cfg=./models/yolov5x.yaml, data=/content/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=True, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=grain_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-178-g799e3d0 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, io

In [ ]:
!python detect.py --weights ./runs/train/grain_results4/weights/best.pt --img 640 --conf 0.5 --source '/content/dataset/test4.jpg'